In [1]:
import pandas as pd
import yfinance as yf
from tqdm import tqdm


esg_csv = "/workspaces/ginappedrosa_project_test/sp500_esg_ceo_info-filtered.csv"  
esg_df = pd.read_csv(esg_csv)

print("Columnas CSV ESG:", esg_df.columns)
print("Número de tickers en CSV:", esg_df["Ticker"].nunique())

Columnas CSV ESG: Index(['Ticker', 'ESG Score', 'Governance Score', 'Environment Score',
       'Social Score', 'ESG Score Date', 'ESG Status', 'CEO Full Name',
       'CEO Gender', 'CEO Status', 'Year'],
      dtype='object')
Número de tickers en CSV: 426


In [ ]:
# Selección de únicamente 50 tickers del CSV sp500_esg_ceo_info-filtered

tickers = esg_df["Ticker"].dropna().unique().tolist()[:50]

start_date = "2018-01-01"
end_date = "2025-08-30"

all_data = []


In [ ]:
batch_size = 50
for i in range(0, len(tickers), batch_size):
    batch = tickers[i:i+batch_size]
    print(f"\nDescargando batch {i//batch_size + 1} de {len(tickers)//batch_size + 1}...")
    
    try:
        df = yf.download(
            batch,
            start=start_date,
            end=end_date,
            progress=False,
            auto_adjust=False  # 
        )
        
        # Pasamos de columnas multi-índice a columnas simples
        df = df.stack(level=1).reset_index()
        df.rename(columns={"level_1": "Ticker"}, inplace=True)
        
        all_data.append(df)
    except Exception as e:
        print(f"Error en batch {i//batch_size + 1}: {e}")



Descargando batch 1 de 2...


/tmp/ipykernel_32970/1935151938.py:16: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=1).reset_index()


In [ ]:
# Unimos todos los datos financieros
fin_df = pd.concat(all_data, ignore_index=True)

print(f"\nDatos financieros: {fin_df.shape}")


✅ Datos financieros descargados: (96269, 8)


In [5]:
# Unimos con ESG (por ticker)
dataset_final = pd.merge(fin_df, esg_df, on="Ticker", how="inner")

In [ ]:
# Creamos features de volatilidad para obtener 2 posibles targets (y) para nuestro futuro modelo predictivo

dataset_final["Daily_Return"] = dataset_final.groupby("Ticker")["Adj Close"].pct_change()
dataset_final["Daily_Volatility"] = (
    dataset_final.groupby("Ticker")["Daily_Return"]
    .rolling(5)
    .std()
    .reset_index(0, drop=True)
)

In [ ]:
# Guardar CSV final
dataset_final.to_csv("dataset_final.csv", index=False)

print(f"\nDataset guardado como 'dataset_final.csv'")
print("Shape final:", dataset_final.shape)
print("\nPrimeras filas:\n", dataset_final.head())

dataset_final.info()


🌸 Dataset guardado como 'dataset_final.csv'
Shape final: (96269, 20)

Primeras filas:
         Date Ticker  Adj Close      Close       High        Low       Open  \
0 2018-01-02      A  63.909588  67.599998  67.889999  67.339996  67.419998   
1 2018-01-02    AAL  51.647564  52.990002  53.099998  51.900002  52.330002   
2 2018-01-02   AAPL  40.381001  43.064999  43.075001  42.314999  42.540001   
3 2018-01-02   ABBV  70.405540  98.410004  98.900002  96.750000  97.139999   
4 2018-01-02    ABT  51.307526  58.790001  59.200001  57.820000  58.200001   

        Volume  ESG Score  Governance Score  Environment Score  Social Score  \
0    1047800.0      15.24              6.26               0.34          8.65   
1    4084700.0      29.10              5.26              11.53         12.31   
2  102223600.0      16.68              9.18               0.65          6.86   
3    4561300.0      27.84              9.95               1.12         16.77   
4   10112800.0      24.98              8.36

EDA

In [9]:
dataset_final.head()

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,ESG Score,Governance Score,Environment Score,Social Score,ESG Score Date,ESG Status,CEO Full Name,CEO Gender,CEO Status,Year,Daily_Return,Daily_Volatility
0,2018-01-02,A,63.909588,67.599998,67.889999,67.339996,67.419998,1047800.0,15.24,6.26,0.34,8.65,01/01/2023,success,Mr. Michael R. McMullen,male,success,23,NaN,NaN
1,2018-01-02,AAL,51.647564,52.990002,53.099998,51.900002,52.330002,4084700.0,29.10,5.26,11.53,12.31,01/01/2023,success,Mr. Robert D. Isom Jr.,male,success,23,NaN,NaN
2,2018-01-02,AAPL,40.381001,43.064999,43.075001,42.314999,42.540001,102223600.0,16.68,9.18,0.65,6.86,01/08/2022,success,Mr. Timothy D. Cook,male,success,22,NaN,NaN
3,2018-01-02,ABBV,70.405540,98.410004,98.900002,96.750000,97.139999,4561300.0,27.84,9.95,1.12,16.77,01/08/2022,success,Mr. Richard A. Gonzalez,male,success,22,NaN,NaN
4,2018-01-02,ABT,51.307526,58.790001,59.200001,57.820000,58.200001,10112800.0,24.98,8.36,3.03,13.59,01/08/2022,success,Mr. Robert B. Ford,male,success,22,NaN,NaN


In [8]:
dataset_final.shape

(96269, 20)

In [10]:
dataset_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96269 entries, 0 to 96268
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               96269 non-null  datetime64[ns]
 1   Ticker             96269 non-null  object        
 2   Adj Close          96269 non-null  float64       
 3   Close              96269 non-null  float64       
 4   High               96269 non-null  float64       
 5   Low                96269 non-null  float64       
 6   Open               96269 non-null  float64       
 7   Volume             96269 non-null  float64       
 8   ESG Score          96269 non-null  float64       
 9   Governance Score   96269 non-null  float64       
 10  Environment Score  96269 non-null  float64       
 11  Social Score       96269 non-null  float64       
 12  ESG Score Date     96269 non-null  object        
 13  ESG Status         96269 non-null  object        
 14  CEO Fu

In [11]:
dataset_final.drop_duplicates()

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,ESG Score,Governance Score,Environment Score,Social Score,ESG Score Date,ESG Status,CEO Full Name,CEO Gender,CEO Status,Year,Daily_Return,Daily_Volatility
0,2018-01-02,A,63.909588,67.599998,67.889999,67.339996,67.419998,1047800.0,15.24,6.26,0.34,8.65,01/01/2023,success,Mr. Michael R. McMullen,male,success,23,NaN,NaN
1,2018-01-02,AAL,51.647564,52.990002,53.099998,51.900002,52.330002,4084700.0,29.10,5.26,11.53,12.31,01/01/2023,success,Mr. Robert D. Isom Jr.,male,success,23,NaN,NaN
2,2018-01-02,AAPL,40.381001,43.064999,43.075001,42.314999,42.540001,102223600.0,16.68,9.18,0.65,6.86,01/08/2022,success,Mr. Timothy D. Cook,male,success,22,NaN,NaN
3,2018-01-02,ABBV,70.405540,98.410004,98.900002,96.750000,97.139999,4561300.0,27.84,9.95,1.12,16.77,01/08/2022,success,Mr. Richard A. Gonzalez,male,success,22,NaN,NaN
4,2018-01-02,ABT,51.307526,58.790001,59.200001,57.820000,58.200001,10112800.0,24.98,8.36,3.03,13.59,01/08/2022,success,Mr. Robert B. Ford,male,success,22,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96264,2025-08-29,GOOGL,212.719742,212.910004,214.649994,210.199997,210.509995,39728400.0,24.14,11.39,1.66,11.10,01/08/2022,success,Mr. Sundar Pichai,male,success,22,0.006001,0.010028
96265,2025-08-29,LNT,65.070000,65.070000,65.459999,64.750000,65.099998,1669600.0,21.87,6.18,10.04,5.65,01/01/2023,success,Mr. John O. Larsen,male,success,23,0.000769,0.006497
96266,2025-08-29,MMM,155.529999,155.529999,158.240005,155.320007,157.600006,3279100.0,33.61,7.72,12.33,13.56,01/08/2022,success,Mr. Michael F. Roman,male,success,22,-0.012884,0.009568
96267,2025-08-29,MO,67.209999,67.209999,67.250000,66.459999,66.550003,5930600.0,23.99,6.35,4.66,12.98,01/08/2022,success,Mr. William F. Gifford Jr.,male,success,22,0.011894,0.008617


In [12]:
dataset_final.dropna(inplace=True)

In [13]:
dataset_final.shape

(96019, 20)

In [ ]:
tickers_only = dataset_final[['Ticker']].drop_duplicates()  # eliminamos duplicados de los tickers para obtener los nombres exactos de las empresas elegidas

tickers_only.to_csv("tickers_only.csv", index=False)
